<a href="https://colab.research.google.com/github/kevinfreire/Speed_Prediction/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
mkdir Project

In [1]:
cd /content/Project

/content/Project


In [3]:
!git clone https://github.com/commaai/speedchallenge.git

Cloning into 'speedchallenge'...
remote: Enumerating objects: 11, done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (11/11), done.


In [2]:
cd speedchallenge/data/

/content/Project/speedchallenge/data


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!cp -r '/content/gdrive/My Drive/Neural_Networks/images' /content/Project/speedchallenge/data

In [7]:
!cp -r '/content/gdrive/My Drive/Neural_Networks/train_images' /content/Project/speedchallenge/data

I had to copy my images and train_images folder from google drive because my code would not work in google colab.  Only code running is to get my labels for the training images.

In [8]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt

# The video feed is read in as a VideoCapture object
cap = cv.VideoCapture('train.mp4')

#for frame identity
def extractImages(nFrames):

    # Creates new file for extracted images from video
    if not os.path.exists('images'):
        os.makedirs('images')

    index = 0
    while(index < nFrames):
        # ret = a boolean return value from getting the frame, frame = the current frame being projected in the video
        ret, frame = cap.read()
        # Converts each frame to grayscale - we previously only converted the first frame to grayscale
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

        # Saves images
        name = './images/frame' + str(index) + '.jpg'
        cv.imwrite(name, gray)

        # next frame
        index += 1

def newTrainImages(space, index, nFrames):

    if not os.path.exists('train_images'):
        os.makedirs('train_images')

    while index<nFrames:
        img1_name = './images/frame' + str(index-space) + '.jpg'
        img2_name = './images/frame' + str(index+space) + '.jpg'
        img1 = cv.imread(img1_name, 0)
        img2 = cv.imread(img2_name, 0)

        # Subtracting both images
        diff = cv.absdiff(img1, img2)

        # Set every pixel that changed by 40 to 255, and all the others to zero
        thershold_value = 40
        set_to_value = 255
        ret, thresh = cv.threshold(diff, thershold_value, set_to_value, cv.THRESH_BINARY)

        # Saves images
        diff_name = './train_images' + '/frame_' + str(index-space) + '_' + str(index+space) + '.jpg'
        print ('Creating...' + diff_name)
        cv.imwrite(diff_name, thresh)

        # next frame subtraction
        index += 2*space

        if index>nFrames:
            break

def getLabels(space, index, nFrames):
    
    with open("train.txt") as train:
        with open("target.txt","w") as target:
            for line in train.read().split("\n")[space::2*space]:
                target.write(line)
                target.write("\n")
                index += 2*space
                if index+2*space>nFrames:
                    break

nFrames = 20400
space = 3
index = 3

# extractImages(nFrames)
# newTrainImages(space, index, nFrames)
getLabels(space, index, nFrames)

cap.release()
cv.destroyAllWindows()

In [ ]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import numpy as np
from numpy import asarray
import PIL
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

RUN_FULL = 0

def get_data(path, input_shape, limit):
    
    x_data = []
    y_data = []
    i=0
    # Iterate through the names of contents of the folder
    for img in os.listdir(path):
        img = cv.imread(path+img)
        x_data.append(np.resize(img, input_shape))
        if RUN_FULL:
          i += 1
          if(i==limit):
              break

    x_data = np.asarray(x_data)

    i=0
    with open("target.txt") as target:
        for line in target:
            y_data.append(tf.strings.to_number(line.rstrip()))
            if RUN_FULL:
              i += 1
              if i==limit:
                  break

    y_data = np.asarray(y_data)

    # spliting data between train (60%) and test (40%) sets
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.5, shuffle=False)

    return (x_train, x_test, y_train, y_test)

##################################################### MODEL ##############################################################
### TPU setup
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))


# Retrieving Data
limit = 1000
input_shape = (224, 224, 3)
path = "./train_images/"
(x_train, x_test, y_train, y_test) = get_data(path, input_shape, limit)

# We build the base model
base_model = ResNet152(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
# base_model = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape)

# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers: 
    layer.trainable = False
    print('Layer ' + layer.name + ' frozen.')

# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='linear', name='predictions')(x)
model = Model(base_model.input, x)

# We compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='mean_squared_error', metrics=['accuracy'])

# Retrieve Model summary
# model.summary()

# We start the training
epochs = 5
batch_size = 32

# We train it
model.fit(x=x_train, y=y_train,
          batch_size=batch_size,
          validation_data=(x_test, y_test),
          epochs=epochs)

# We evaluate the accuracy and the loss in the test set
scores = model.evaluate(x_test, y_test, verbose=1)

model.save("predict_speed.model", save_format="h5")
print("finished saving model")
print('Test loss:', scores[0])

######################################## Testing for accuracy #########################################

# Load model
model = load_model('predict_speed.model')

# We predict the output of the image test set
scores = model.predict(x_test, verbose=1)

# Calculating accuracy with +/-2 difference
count=0
for i in range(len(y_test)):
    if(y_test[i]-2 < scores[i] and scores[i] < y_test[i]+2):
        count += 1

accuracy = (count / len(y_test)) * 100
print("Accuracy: " + str(accuracy) + "%")
exit()

INFO:tensorflow:Initializing the TPU system: grpc://10.8.130.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.8.130.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]
Layer input_2 frozen.
Layer conv1_pad frozen.
Layer conv1_conv frozen.
Layer conv1_bn frozen.
Layer conv1_relu frozen.
Layer pool1_pad frozen.
Layer pool1_pool frozen.
Layer conv2_block1_1_conv frozen.
Layer conv2_block1_1_bn frozen.
Layer conv2_block1_1_relu frozen.
Layer conv2_block1_2_conv frozen.
Layer conv2

23/54 [===========>..................] - ETA: 20s - loss: 118.8531 - accuracy: 0.0000e+00

In [7]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array, array_to_img
import numpy as np
from numpy import asarray
import PIL
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def get_data(path, input_shape, limit):
    
    x_data = []
    y_data = []
    i=0
    # Iterate through the names of contents of the folder
    for img in os.listdir(path):
        img = cv.imread(path+img)
        x_data.append(np.resize(img, input_shape))
        i += 1
        if(i==limit):
            break

    # Splitting x_data to smaller dataset due to lack of memory
    x_data = np.asarray(x_data)
    x_reduce = x_data[0:100]

    i=0
    with open("target.txt") as target:
        for line in target:
            y_data.append(tf.strings.to_number(line.rstrip()))
            i += 1
            if i==limit:
                break

    y_data =np.asarray(y_data)
    # Slicing y_data to smaller dataset due to lack of memory
    y_reduce = y_data[0:100]

    # spliting data between train (60%) and test (40%) sets
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.5, shuffle=False)

    return (x_train, x_test, y_train, y_test)

### TPU setup
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

# Retrieving Data
limit = 500
input_shape = (224, 224, 3)
path = "./train_images/"
(x_train, x_test, y_train, y_test) = get_data(path, input_shape, limit)

# Load model
model = load_model('predict_speed.model')

# We evaluate the accuracy and the loss in the test set
scores = model.predict(x_test, verbose=1)

# Calculating accuracy with +/- 2 difference
count=0
for i in range(len(y_test)):
    if(y_test[i]-2 < scores[i] and scores[i] < y_test[i]+2):
        count += 1

accuracy = (count / len(y_test)) * 100
print(scores)
print("Accuracy: " + str(accuracy) + "%")

INFO:tensorflow:Initializing the TPU system: grpc://10.8.130.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.8.130.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]
2/8 [======>.......................] - ETA: 3sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0027s vs `on_predict_batch_end` time: 0.5551s). Check your callbacks.


8/8 [==============================] - 5s 617ms/step
[[11.61753  ]
 [11.141864 ]
 [11.61753  ]
 [15.368801 ]
 [15.190458 ]
 [11.61753  ]
 [16.034561 ]
 [13.28047  ]
 [13.525353 ]
 [11.61753  ]
 [13.285424 ]
 [13.613017 ]
 [10.357581 ]
 [15.52226  ]
 [11.61753  ]
 [13.554738 ]
 [12.891431 ]
 [14.386001 ]
 [11.61753  ]
 [14.827634 ]
 [17.107805 ]
 [13.995845 ]
 [17.213257 ]
 [11.487732 ]
 [15.110612 ]
 [11.61753  ]
 [16.244413 ]
 [13.012201 ]
 [13.089238 ]
 [11.61753  ]
 [13.123799 ]
 [11.61753  ]
 [11.61753  ]
 [14.924037 ]
 [14.571889 ]
 [11.255042 ]
 [16.412123 ]
 [13.258803 ]
 [14.020982 ]
 [16.51767  ]
 [11.61753  ]
 [19.580288 ]
 [20.213669 ]
 [11.61753  ]
 [ 9.8770485]
 [14.436654 ]
 [13.107366 ]
 [11.61753  ]
 [14.339296 ]
 [11.61753  ]
 [13.604737 ]
 [11.61753  ]
 [11.665123 ]
 [11.61753  ]
 [11.61753  ]
 [18.914223 ]
 [17.780293 ]
 [13.848376 ]
 [17.093525 ]
 [17.88473  ]
 [16.470003 ]
 [19.986603 ]
 [16.647903 ]
 [23.494577 ]
 [11.61753  ]
 [17.241333 ]
 [17.409403 ]
 [21.3970